In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [5]:
!git clone https://github.com/mk-minchul/AdaFace.git

fatal: destination path 'AdaFace' already exists and is not an empty directory.


In [6]:
!pip install pytorch-lightning

In [7]:
import sys
sys.path.append('/content/AdaFace')
sys.path.append('/content/vgg_face2')

In [8]:

import net
import torch
import os
from face_alignment import align
import numpy as np


adaface_models = {
    'ir_50':"/content/drive/MyDrive/Pre-trained models/adaface_ir50_ms1mv2.ckpt",
}

def load_pretrained_model(architecture='ir_50'):
    # load model and pretrained statedict
    assert architecture in adaface_models.keys()
    model = net.build_model(architecture)
    statedict = torch.load(adaface_models[architecture])['state_dict']
    model_statedict = {key[6:]:val for key, val in statedict.items() if key.startswith('model.')}
    model.load_state_dict(model_statedict)
    model.eval()
    return model

def to_input(pil_rgb_image):
    np_img = np.array(pil_rgb_image)
    brg_img = ((np_img[:,:,::-1] / 255.) - 0.5) / 0.5
    # tensor = torch.tensor([brg_img.transpose(2,0,1)]).float()
    return torch.tensor(brg_img).float()


In [9]:
# model = load_pretrained_model('ir_50')
# feature, norm = model(torch.randn(2,3,112,112))

# test_image_path = '/content/drive/MyDrive/Dataset/dataset/train'
# all_features = []
# for fname in sorted(os.listdir(test_image_path)):
#     path = os.path.join(test_image_path, fname)
#     features = []
#     for img in sorted(os.listdir(path)):
#       img_path = os.path.join(path, img)
#       print(img_path)
#       aligned_rgb_img = align.get_aligned_face(img_path)
#       bgr_tensor_input = to_input(aligned_rgb_img)
#       feature, _ = model(bgr_tensor_input)
#       features.append(feature)
#     all_features.append(features)

In [10]:
from sklearn.datasets import fetch_lfw_pairs

lfw_pairs_train = fetch_lfw_pairs(subset='train')

In [11]:
lfw_pairs_train.pairs.shape

(2200, 2, 62, 47)

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from torchvision import transforms
# from adafaces import AdaFace
adafaces_model = load_pretrained_model('ir_50')

# Load LFW dataset
lfw_people = fetch_lfw_people(min_faces_per_person=70, color=True)
X, y = lfw_people.images, lfw_people.target

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_embeddings = []
X_test_embeddings = []

for i in range(len(X_train)):
  img = torch.Tensor(X_train[i])
  img = img.permute(2,0,1).unsqueeze(0)
  output_tensor = F.interpolate(img, size=(112, 112), mode='bilinear', align_corners=False)
  # print(output_tensor.shape)
  output_tensor = output_tensor.permute(0, 2, 3, 1)
  # print(output_tensor.shape)
  input_img = to_input(output_tensor).permute(0, 3, 1, 2)
  img1, _ = adafaces_model(input_img)
  # print(img1.view(-1).shape)
  # print(len(img1.tolist()))
  X_train_embeddings.append(img1.view(-1).tolist())

for i in range(len(X_test)):

  img = torch.Tensor(X_train[i])
  img = img.permute(2,0,1).unsqueeze(0)
  output_tensor = F.interpolate(img, size=(112, 112), mode='bilinear', align_corners=False)
  # print(output_tensor.shape)
  output_tensor = output_tensor.permute(0, 2, 3, 1)
  # print(output_tensor.shape)
  input_img = to_input(output_tensor).permute(0, 3, 1, 2)
  img1, _ = adafaces_model(input_img)

  X_test_embeddings.append(img1.view(-1).tolist())




In [66]:
from sklearn.metrics.pairwise import cosine_similarity
X_train_new = []
X_test_new = []
y_train_new = []
y_test_new = []

for i in range(0,len(X_train_embeddings)):
  for j in range(i, len(X_train_embeddings)):
    X_train_new.append((X_train_embeddings[i], X_train_embeddings[j]))
    y_train_new.append(cosine_similarity(X_train_embeddings[i].reshape(1,-1), X_train_embeddings[j].reshape(1,-1)))


for i in range(0,len(X_test_embeddings)):
  for j in range(i, len(X_test_embeddings)):
    X_test_new.append((X_test_embeddings[i], X_test_embeddings[j]))
    y_test_new.append(cosine_similarity(X_test_embeddings[i].reshape(1,-1), X_test_embeddings[j].reshape(1,-1)))


In [76]:
from torch.utils.data import Dataset, DataLoader

class Embeddings(Dataset):
  def __init__(self, embeddings, similarity):
    self.imgs = embeddings
    self.similarity = similarity

  def __len__(self):
    return len(self.similarity)

  def __getitem__(self, idx):
    img1 = torch.FloatTensor(self.imgs[idx][0])
    img2 = torch.FloatTensor(self.imgs[idx][1])
    sim = torch.FloatTensor(self.similarity[idx])

    return img1, img2, sim

In [77]:
train_data = Embeddings(X_train_new, y_train_new)
train_dataloader = DataLoader(train_data, batch_size=16, shuffle=True)

test_data = Embeddings(X_test_new, y_test_new)
test_dataloader = DataLoader(test_data, batch_size=16, shuffle=True)

In [78]:
labels = set(y_train)
print(len(labels))

1030


In [95]:
from torch.nn.modules import MSELoss
from torch.nn.modules.loss import CrossEntropyLoss
# # Convert to PyTorch tensors
# X_train_embeddings, X_test_embeddings = torch.tensor(X_train_embeddings).float(), torch.tensor(X_test_embeddings).float()
# y_train, y_test = torch.tensor(y_train), torch.tensor(y_test)

# Use AdaFace for face embedding

class CompressionModel(nn.Module):
    def __init__(self, input_dim, compressed_dim):
        super(CompressionModel, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, compressed_dim),
            nn.ReLU()
        )
        # self.linear = nn.Linear(compressed_dim, 1030)

    def forward(self, x):
        encoded = self.encoder(x)
        # y = self.linear(encoded)
        # y = F.softmax(y, dim = 0)
        return encoded

class Loss(nn.Module):
    def __init__(self):
        super(Loss, self).__init__()


    def forward(self, img1, img2, true_sim):
        # Compute cosine similarity loss
        sim_score = F.cosine_similarity(img1, img2)
        # print(true_sim.view(-1).shape)
        loss = MSELoss()(sim_score, true_sim.view(-1))
        # print(loss)
        return loss




In [97]:
input_dim = 512
compressed_dim = 128
model = CompressionModel(input_dim, compressed_dim)

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = Loss()

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
  model.train()
  total_loss = 0
  for img1, img2, sim_score in train_dataloader:
      # print(X_train[i].unsqueeze(0).shape)

      # print(X_train_embeddings[i].shape)
      # Forward pass
      compressed_img1 = model(img1)
      compressed_img2 = model(img2)

      loss = criterion(compressed_img1, compressed_img2, sim_score)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
        # for name, param in model.named_parameters():
        #   if param.requires_grad:
        #     print(name, param.grad)



    # for name, param in model.named_parameters():
    #   if param.requires_grad:
    #     print(name, param.grad)
      total_loss += loss.item()

  print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss}")

print("Training complete!")


Epoch [1/10], Loss: 1.2518103592821378e-06
Epoch [2/10], Loss: 3.008335028196682e-07
Epoch [3/10], Loss: 2.3758208174146783e-07
Epoch [4/10], Loss: 1.9882635706727422e-07
Epoch [5/10], Loss: 1.7292707164004786e-07
Epoch [6/10], Loss: 1.5902935630602438e-07
Epoch [7/10], Loss: 1.4498030767020817e-07
Epoch [8/10], Loss: 1.4571598843087646e-07
Epoch [9/10], Loss: 1.5026597131038207e-07
Epoch [10/10], Loss: 1.396859835799645e-07
Training complete!


In [101]:
model.eval()
true_score = []
pred_score = []
with torch.no_grad():
  for img1, img2, sim_score in test_dataloader:
    compressed_img1 = model(img1)
    compressed_img2 = model(img2)
    true_score.extend(sim_score.numpy())
    pred_score.extend(F.cosine_similarity(compressed_img1, compressed_img2).numpy())


true_score = [(x+1)/2 for x in true_score]
pred_score = [(x+1)/2 for x in pred_score]


In [106]:
print(type(true_score))

<class 'list'>


In [107]:
res = sum(x-y for x,y in zip(true_score,pred_score))
print(res)

[[-0.00740111]]


In [102]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

fpr, tpr, _ = roc_curve(true_score, pred_score)
roc_auc = auc(fpr, tpr)

plt.figure(figsize =(8,6))
plt.plot(fpr, tpr)
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC curve')
plt.show()


ValueError: ignored